# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Output File (CSV)
output_data_file = "output_data/Hotels.csv"

# Import API key
from api_keys import g_key

In [9]:
hotel_df

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,
1,30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,
2,31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24,
3,37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05,
4,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
5,84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93,
6,141,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24,
7,168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17,
8,192,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93,
9,214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93,


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_file= "../output_data/cities.csv"
city_data_df = pd.read_csv(city_data_file)
city_data_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_data_df[['Lat','Lng']]
center_map = (0,0)

fig = gmaps.figure(map_type='ROADMAP')

heatmap_layer = gmaps.heatmap_layer(
    city_data_df[["Lat", "Lng"]], weights=city_data_df["Humidity"],
    max_intensity=100, point_radius=15)
fig = gmaps.figure(layout={'width' : '100%', 'height' : '675px'},center=(0,0),zoom_level=1.9)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='675px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# narrow down weather df
# Max temp <80, Min temp >70, Cloudiness =0, Wind Speed <10mph
best_weather_df=city_data_df[city_data_df["Max Temp"] <75]
best_weather_df=best_weather_df[best_weather_df["Max Temp"] >55]
best_weather_df=best_weather_df[best_weather_df["Wind Speed"] <10]
best_weather_df=best_weather_df[best_weather_df["Cloudiness"]<1]
best_weather_df.dropna()
best_weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93
141,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
192,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93
214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93


## Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = best_weather_df
hotel_df["Hotel Name"] = ""
hotel_df.reset_index(inplace=True)
hotel_df.drop(columns='index')
lat = hotel_df["Lat"].astype(float)
lng = hotel_df["Lng"].astype(float)
hotel_df.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,
1,30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,
2,31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24,
3,37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05,
4,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,


In [6]:
import pprint
lat = hotel_df["Lat"]
lng = hotel_df["Lng"]
hotels=[] 

for i in range (len(hotel_df)):
    target_type = "lodging"
    location = f"{lat[i]},{lng[i]}"
    keyword = "hotel"
    radius = 5000
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params= {
    "location" : location,
    "radius" : radius,
    "types" : target_type,
    "key" : g_key
    }
    
    try:
        response = requests.get(base_url, params=params).json()
        pprint(response)
        hotel_name = response
        hotels.append(hotel_name["results"][0]["name"])
        print(hotel_name["results"][0]["name"])
    except:
        hotels.append("")
        print("No result found. Skipping ... ")
        pass

No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 
No result found. Skipping ... 


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='675px', width='100%'))